## Sistema Inteligente para FAQs

### Importa Bibliotecas

In [4]:
import pandas as pd
import json
from bs4 import BeautifulSoup
from sentence_transformers import SentenceTransformer
import numpy as np
import pickle
import os

### Carregar e limpar o Dataset de FAQ

In [5]:
print("Iniciando o carregamento e limpeza do FAQ...")

# Define o caminho para o arquivo JSON
arquivo_json = '..\data\FAQ_BB.json'

try:
    # --- 1.1 Carregando o arquivo JSON ---
    with open(arquivo_json, 'r', encoding='utf-8') as f:
        dados_json = json.load(f)

    # O conteúdo está dentro da chave "conteudo"
    df = pd.DataFrame(dados_json['conteudo'])

    # --- 1.2 Limpando as respostas (removendo HTML) ---
    
    # Função para extrair texto puro do HTML
    def limpar_html(texto_html):
        if not isinstance(texto_html, str):
            return ""  # Retorna string vazia se o valor não for texto
        # Usa BeautifulSoup para analisar o HTML e extrair o texto
        soup = BeautifulSoup(texto_html, "html.parser")
        return soup.get_text(separator=' ', strip=True)

    # Aplica a função de limpeza na coluna 'resposta' para criar uma nova coluna limpa
    df['resposta_limpa'] = df['resposta'].apply(limpar_html)
    
    # --- 1.3 Criando o DataFrame Final ---
    
    # Seleciona apenas as colunas que importam e remove linhas onde a pergunta ou resposta possam estar vazias
    df_faq_limpo = df[['pergunta', 'resposta_limpa']].copy()
    df_faq_limpo.dropna(inplace=True)
    df_faq_limpo = df_faq_limpo[df_faq_limpo['pergunta'] != '']

    # Renomeia a coluna de resposta limpa para um nome mais simples
    df_faq_limpo.rename(columns={'resposta_limpa': 'resposta'}, inplace=True)
    
    print("\n✅ Limpeza concluída com sucesso!")
    print(f"Total de pares de Pergunta/Resposta limpos: {len(df_faq_limpo)}")
    
    print("\nInformações do DataFrame final:")
    df_faq_limpo.info()

    print("\nExemplo dos dados após a limpeza:")
    print(df_faq_limpo.head())
    
except FileNotFoundError:
    print(f"❌ ERRO: O arquivo '{arquivo_json}' não foi encontrado. Verifique o caminho.")
except Exception as e:
    print(f"❌ Ocorreu um erro inesperado: {e}")

Iniciando o carregamento e limpeza do FAQ...

✅ Limpeza concluída com sucesso!
Total de pares de Pergunta/Resposta limpos: 1172

Informações do DataFrame final:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1172 entries, 0 to 1171
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   pergunta  1172 non-null   object
 1   resposta  1172 non-null   object
dtypes: object(2)
memory usage: 18.4+ KB

Exemplo dos dados após a limpeza:
                                            pergunta  \
0                                O que é Registrato?   
1             Quais são os relatórios do Registrato?   
2  Os relatórios do Registrato são atualizados at...   
3  Recebi um relatório do Registrato incorreto, c...   
4                          Como acesso o Registrato?   

                                            resposta  
0  O Registrato é um sistema onde você pode consu...  
1  Relatório de Chaves Pix, Relatório de Emprésti...  
2  

### Vetorização da Base de Conhecimento 

In [6]:
print("\n--- Iniciando a vetorização das perguntas ---")

# --- 2.1 Carregando o modelo pré-treinado ---
# Este modelo é ótimo para português e transforma sentenças em vetores de 768 dimensões.
try:
    model = SentenceTransformer('distiluse-base-multilingual-cased-v1')
    print("✅ Modelo SentenceTransformer carregado com sucesso.")
except Exception as e:
    print(f"❌ ERRO ao carregar o modelo SentenceTransformer: {e}")

# --- 2.2 Convertendo as perguntas em vetores (embeddings) ---
# Pega a lista de perguntas do nosso dataframe limpo
perguntas = df_faq_limpo['pergunta'].tolist()

print("\nGerando embeddings para as perguntas... (Isso pode levar alguns minutos)")
embeddings_perguntas = model.encode(perguntas, show_progress_bar=True)
print("✅ Embeddings gerados com sucesso!")
print(f"Formato da matriz de embeddings: {embeddings_perguntas.shape}")

# --- 2.3 Salvando os artefatos para a aplicação Flask ---
# Vamos criar uma pasta 'models' para guardar nossos arquivos
ARTEFATOS_DIR = '..\models'
if not os.path.exists(ARTEFATOS_DIR):
    os.makedirs(ARTEFATOS_DIR)

# Salva a matriz de embeddings
np.save(os.path.join(ARTEFATOS_DIR, 'embeddings_faq.npy'), embeddings_perguntas)

# Salva a lista de perguntas e respostas para consulta futura
with open(os.path.join(ARTEFATOS_DIR, 'dados_faq.pkl'), 'wb') as f:
    pickle.dump({
        'perguntas': perguntas,
        'respostas': df_faq_limpo['resposta'].tolist()
    }, f)

print(f"\n✅ Artefatos salvos com sucesso na pasta '{ARTEFATOS_DIR}'")


--- Iniciando a vetorização das perguntas ---
✅ Modelo SentenceTransformer carregado com sucesso.

Gerando embeddings para as perguntas... (Isso pode levar alguns minutos)


Batches: 100%|█████████████████████████████████████████████████████████████████████████| 37/37 [00:35<00:00,  1.06it/s]

✅ Embeddings gerados com sucesso!
Formato da matriz de embeddings: (1172, 512)

✅ Artefatos salvos com sucesso na pasta '..\models'
